In [2]:
from pymongo import MongoClient
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
%matplotlib inline

## Connecting to existing database and table within the database

In [3]:
client = MongoClient()

In [4]:

db = client.images2
collection= db.image_data

In [5]:
print db
print collection

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), u'images2')
Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), u'images2'), u'image_data')


## Test connection

In [6]:
#Testing retreving data from Mongodb. Imp for when an img-no doesnt exist
cur = collection.find({'img_no':29},{'_id':0,'img_no':1,'prod_info_table':1,'prod_desc':1,"title":1})
cur.rewind()

try:
    print (cur)[0]['title']
except:
    print "Not readable error"
    

Invicta Men's 8932 Pro Diver Collection Silver-Tone Watch


In [7]:
cur[0]['prod_desc']

u'Big, bold and masculine, the Invicta Quartz Pro Diver 8932 is an affordable luxury watch with classically styled features. This precisely constructed quartz timepiece for men features a solid stainless steel case that\'s 37mm wide, slightly smaller than its 8926 cousin. It has a black face with Tritnite luminous hands and markers and a unidirectional black bezel with white numerals and markings. Other features include anti-reflective mineral crystal, date display at 3 o\'clock, and water resistance to 200 meters. The highly polished stainless steel band has a diver buckle with safety clasp.Pro Diver Collection Plunge into any horizon using the steadfast guidance of the Invicta Pro Diver. Stylishly classic, internal workings are forged with variations of either Swiss chronograph or 21-jewel automatic movements and willingly navigate in depths up to 300 meters. Built with confident prowess, the fortitude with which these timepieces function makes the Pro Diver the quintessential in per

## Getting the information from MongoDB

In [8]:
#total number of docs in collections
total_docs = collection.count()

In [9]:
#getting the data(img_no and prod_info_table) out of Mongodb
#5940 is the max img_no known through looking at data
prod_desc_table = []

for i in xrange(5940):
    cursor = collection.find({'img_no':i},{'_id':0,'img_no':1,'prod_info_table':1,'prod_desc':1,'title':1})
    #print cursor[0]
    cursor.rewind()
    try: 
        #We do cursor of 0 because cursor only has 1 result a dictionary. 
        #We are adding the img_no as another feature in the prod_info_table dictionary
        #This is to keep track of the image number for each row . Will be used in retirving the results when we know th
        #nearest neigbours for the model. We also add product description and title for future TFIDF
        #print cursor[0]    
        prod_dict = cursor[0]["prod_info_table"]
        
        #Insert the three in the exsiting dictionary to get one comprehensive dict for dataframe
        prod_dict['Img_no'] = cursor[0]["img_no"]
        prod_dict['Prod_description'] = cursor[0]["prod_desc"]
        prod_dict['Prod_title'] = cursor[0]["title"]
        prod_desc_table.append(prod_dict)
    
        
    except:
        pass # Some number might not exist in between due to broken links while scraping. 
        #This will ensure we move to the next image number without raising an error in that case

In [10]:
len(prod_desc_table)

4957

##  Pandas dataframe and EDA

### Loading dictionary to pandas dataframe

In [11]:
df = pd.DataFrame(prod_desc_table)

In [12]:
len(df.columns)

29

In [13]:
df.columns

Index([                       u'Band Color',
                           u'Band Material',
                             u'Band length',
                              u'Band width',
                     u'Battery description',
                          u'Bezel material',
       u'Brand, Seller, or Collection Name',
                          u'Case Thickness',
                           u'Case diameter',
                           u'Case material',
                                   u'Clasp',
                              u'Dial color',
                            u'Display Type',
                                  u'Height',
                                  u'Img_no',
                             u'Item Length',
                              u'Item Shape',
                             u'Item weight',
                                  u'Length',
                                u'Material',
                             u'Metal stamp',
                              u'Model Year',
          

In [14]:
df.iloc[:,:10].head()

,Band Color,Band Material,Band length,Band width,Battery description,Bezel material,"Brand, Seller, or Collection Name",Case Thickness,Case diameter,Case material
0,Black,Resin,Men's Standard,22 millimeters,NaN,Resin,Casio,12 millimeters,43 millimeters,Resin
1,Black,Calfskin,Mens Standard,18 millimeters,NaN,Brass,Timex,8 millimeters,35 millimeters,Brass
2,Black,Resin,mens,18 millimeters,NaN,Plastic,Casio,8 millimeters,35 millimeters,Plastic
3,Black,Resin,mens,20 millimeters,NaN,Resin,Casio,7.8 millimeters,35 millimeters,Resin
4,Biege,Nylon,Mens-Standard,20 millimeters,NaN,Stainless Steel,Casio,11 millimeters,41 millimeters,Resin


In [15]:
df.iloc[:,10:20].head()

,Clasp,Dial color,Display Type,Height,Img_no,Item Length,Item Shape,Item weight,Length,Material
0,Buckle,Black,Analog,NaN,0,NaN,Round,2.08 Ounces,NaN,NaN
1,Buckle,White,Analog,NaN,1,NaN,Round,1.44 Ounces,NaN,NaN
2,Buckle,White,Analog,NaN,2,NaN,Round,18.14 Grams,NaN,NaN
3,Buckle,White,Analog,NaN,3,NaN,Round,NaN,NaN,NaN
4,Buckle,Beige,Analog,NaN,4,NaN,Round,16 Ounces,NaN,NaN


In [16]:
df.iloc[:,20:].head()

,Metal stamp,Model Year,Model number,Part Number,Prod_description,Prod_title,Special features,Water pressure resistance,Water resistant depth
0,NaN,2014,MRW200H-1BV,MRW200H-1BV,This watch from Casio is water resistant up to...,Casio Men's MRW200H-1BCT,Water Resistant,NaN,330 Feet
1,NaN,2009,T2H281,T2H2819J,Not available,Timex Men's Easy Reader Watch,"measures-seconds, Light",NaN,99 Feet
2,NaN,2014,MQ24-7B2,MQ24-7B2,Splash Resistant / Round White Face / Black Re...,Casio Men's MQ24-7B2 Analog Watch with Black R...,Water Resistant,NaN,NaN
3,NaN,NaN,MQ24-7B,MQ24-7B,The basic black-and-white design of the Casio ...,Casio Unisex MQ24-7B Analog Black Resin Watch,Water Resistant,NaN,50
4,NaN,2014,FT-500WC-5BVCF,FT-500WC-5BVCF,This watch from the Forester collection featur...,Casio Men's FT500WVB-5BV,Luminous,NaN,330 Feet


### Cleaning data

In [17]:
df.isnull().sum(axis =0)

Band Color                             83
Band Material                          33
Band length                           197
Band width                             72
Battery description                  4956
Bezel material                        331
Brand, Seller, or Collection Name       0
Case Thickness                        105
Case diameter                          29
Case material                         119
Clasp                                1768
Dial color                             21
Display Type                           72
Height                               4956
Img_no                                  0
Item Length                          4955
Item Shape                             73
Item weight                          2617
Length                               4956
Material                             4955
Metal stamp                          4460
Model Year                           1780
Model number                           34
Part Number                       

In [18]:
# Anything with more than 50% null values is dropped immediately 
df.drop(['Battery description','Water pressure resistance','Material','Height','Item Length','Metal stamp','Length','Item weight'],axis =1,inplace = True)

In [19]:
df.shape

(4957, 21)

In [20]:
df.head(2)

,Band Color,Band Material,Band length,Band width,Bezel material,"Brand, Seller, or Collection Name",Case Thickness,Case diameter,Case material,Clasp,...,Display Type,Img_no,Item Shape,Model Year,Model number,Part Number,Prod_description,Prod_title,Special features,Water resistant depth
0,Black,Resin,Men's Standard,22 millimeters,Resin,Casio,12 millimeters,43 millimeters,Resin,Buckle,...,Analog,0,Round,2014,MRW200H-1BV,MRW200H-1BV,This watch from Casio is water resistant up to...,Casio Men's MRW200H-1BCT,Water Resistant,330 Feet
1,Black,Calfskin,Mens Standard,18 millimeters,Brass,Timex,8 millimeters,35 millimeters,Brass,Buckle,...,Analog,1,Round,2009,T2H281,T2H2819J,Not available,Timex Men's Easy Reader Watch,"measures-seconds, Light",99 Feet


In [21]:
#Remove Model number and part number beacuase they arent giving us useful info(plus already in title)
df.drop(['Model number','Part Number'],axis =1,inplace = True)

In [22]:
print df.columns
df.shape

Index([                       u'Band Color',
                           u'Band Material',
                             u'Band length',
                              u'Band width',
                          u'Bezel material',
       u'Brand, Seller, or Collection Name',
                          u'Case Thickness',
                           u'Case diameter',
                           u'Case material',
                                   u'Clasp',
                              u'Dial color',
                            u'Display Type',
                                  u'Img_no',
                              u'Item Shape',
                              u'Model Year',
                        u'Prod_description',
                              u'Prod_title',
                        u'Special features',
                   u'Water resistant depth'],
      dtype='object')


(4957, 19)

We still have plenty of null values but we'll deal with them with them as we go along.
Since we apply Tfidf to categorical part we'll start by dealing with numerical data seperately. We will later combine the data together to form a feature space   


In [102]:
dfnum = df[['Band width','Water resistant depth','Case diameter','Case Thickness','Img_no']]
dfcat = df.drop(['Band width','Water resistant depth','Case diameter','Case Thickness'],axis=1)

### Converting all the numerical features into numbers




In [103]:
#There are 5 columns with numbers one of which can be ignored(Img-no). 
#Filling na with ((mean of its values) mm, ounces,feet )Looking for the different numerical units in each of them
def fill_nulls(col_name,units):
    '''
    Input: Column name and units to be added
    Output: None
    '''
    print "No of nulls before filling for %s are"%col_name, sum(dfnum[col_name].isnull())
    numbers = [ float(str(string).split()[0]) for string in dfnum[col_name]]
    
    mean_of_column = np.nanmean(numbers)
    
    value = '{0} {1}'.format(int(mean_of_column),units)
    print ">>>>>>", value
    
    dfnum[col_name].fillna(value, inplace = True )
    
    print "No of nulls now",sum(dfnum[col_name].isnull())
    

In [104]:
fill_nulls("Band width", 'millimeters')
fill_nulls("Case Thickness", 'millimeters')
fill_nulls("Case diameter", 'millimeters')

No of nulls before filling for Band width are 72
>>>>>> 22 millimeters
No of nulls now 0
No of nulls before filling for Case Thickness are 105
>>>>>> 12 millimeters
No of nulls now 0
No of nulls before filling for Case diameter are 29
>>>>>> 40 millimeters
No of nulls now 0


In [105]:
fill_nulls('Water resistant depth','feet')
dfnum.head()

No of nulls before filling for Water resistant depth are 267
>>>>>> 289 feet
No of nulls now 0


,Band width,Water resistant depth,Case diameter,Case Thickness,Img_no
0,22 millimeters,330 Feet,43 millimeters,12 millimeters,0
1,18 millimeters,99 Feet,35 millimeters,8 millimeters,1
2,18 millimeters,289 feet,35 millimeters,8 millimeters,2
3,20 millimeters,50,35 millimeters,7.8 millimeters,3
4,20 millimeters,330 Feet,41 millimeters,11 millimeters,4


In [107]:
#Ensuring that all units are in the same case(lower).Others are already lowercase

dfnum['Water resistant depth'] = map(lambda x:str(x).lower(),dfnum['Water resistant depth'])

dfnum.head(5)

/Users/Iskandar/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


,Band width,Water resistant depth,Case diameter,Case Thickness,Img_no
0,22 millimeters,330 feet,43 millimeters,12 millimeters,0
1,18 millimeters,99 feet,35 millimeters,8 millimeters,1
2,18 millimeters,289 feet,35 millimeters,8 millimeters,2
3,20 millimeters,50,35 millimeters,7.8 millimeters,3
4,20 millimeters,330 feet,41 millimeters,11 millimeters,4


In [108]:
# Function to get unique units per column to study them
'''
Input: Column Name
Output: Returns unique values of units used

'''

def get_unique_num_units(col_name):
    units = []
    for x in dfnum[col_name]:
        try :
            units.append( str(x).split()[1])
        except:
            pass
    u =set(units)
    return u

In [109]:
print get_unique_num_units("Band width")
print get_unique_num_units("Water resistant depth")

set(['decimeters', 'centimeters', 'inches', 'millimeters'])
set(['feet', 'centimeters', 'meters', 'millimeters'])


#### Getting the unit conversions in order

In [110]:
from pint import UnitRegistry

In [111]:
ureg = UnitRegistry()

In [112]:
def unit_conversion_pinthelper(ureg,elem, units_to):
    '''
    Takes a single entry as input and converts it to desired units. If the element doesnt have units,
    except adds the desired units to it.
    
    Input: UnitRegistery object ,column name, units to convert to(an assumption made viewing the data)
    
    Output: A numpy array of data type quantity that can replace the entire 
    
    '''
    try: 
        elem = ureg.Quantity(elem).to(units_to)
    except:
        elem = ureg.Quantity(elem, units_to)
    return  elem

def unit_conversion(colname,desired_units):
    """
    Takes an entire column and changes the units for each of its members by calling unit_conversion_pinthelper
    """
    
    dfnum[colname] = map(lambda x:float(unit_conversion_pinthelper(ureg,x,desired_units).magnitude),dfnum[colname])
    

example of conversion
df["Band width"] = map(lambda x:float(unit_conversion_pinthelper(ureg,x,"millimeter").magnitude),df["Band width"])

In [113]:
unit_conversion("Band width","millimeter")
unit_conversion("Case diameter","millimeter")   
unit_conversion("Case Thickness","millimeter")


/Users/Iskandar/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [114]:
unit_conversion('Water resistant depth',"feet")

/Users/Iskandar/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [115]:
dfnum.head()

,Band width,Water resistant depth,Case diameter,Case Thickness,Img_no
0,22,330,43,12.0,0
1,18,99,35,8.0,1
2,18,289,35,8.0,2
3,20,50,35,7.8,3
4,20,330,41,11.0,4


With this our numerical varibales are ready. Now for categorical variables

## Dealing with categorical variables

In [116]:
dfcat.dtypes

Band Color                           object
Band Material                        object
Band length                          object
Bezel material                       object
Brand, Seller, or Collection Name    object
Case material                        object
Clasp                                object
Dial color                           object
Display Type                         object
Img_no                                int64
Item Shape                           object
Model Year                           object
Prod_description                     object
Prod_title                           object
Special features                     object
dtype: object

In [117]:
columns_df = dfcat.drop(['Img_no',"Prod_description"],axis=1)
#Product description column needs encoding beacuse its 
#inon ascii characters that cant be converted to string by str

In [118]:
#Converting all columns into string datatype
for col in columns_df:
    #dfcat[col] = dfcat[col].astype(str) -  Doesnt work coz of Nan's in the data
    dfcat[col] = map(lambda x: str(x).lower(),dfcat[col])
    print "%s done"%col
# Now we move on to product description tab

Band Color done
Band Material done
Band length done
Bezel material done
Brand, Seller, or Collection Name done
Case material done
Clasp done
Dial color done
Display Type done
Item Shape done
Model Year done
Prod_title done
Special features done


In [119]:
# Since product description has too much unclean data we discard it completely. Also it wasnt helping our predictions
# something we learnt from testing
x = dfcat['Prod_description']
x.shape

(4957,)

In [40]:
# Since non ascii characters don't let us convert the column to string directly
#we need to encode it ignore non ascii. Then its converted to string without issues
#dfcat['Prod_description'] = map(lambda y: str(y.encode('ascii', 'ignore').lower().strip()),dfcat['Prod_description'])
#http://stackoverflow.com/questions/9942594/unicodeencodeerror-ascii-codec-cant-encode-character-u-xa0-in-position-20 

### Getting data ready for TFIDF

In [120]:
#Getting all columns except image number which is a numerical column which should serve as our index
#columns_df = dfcat.drop(['Img_no'],axis=1).columns

In [121]:
#Adding up all the strings
dfcat["Final_String"] = ""
for col in columns_df:
    dfcat["Final_String"] += " "
    dfcat["Final_String"] += dfcat[col] 

In [122]:
print dfcat['Final_String'][8], ">>>>",dfcat["Prod_description"][8]
    

 brown canvas mens stainless steel casio stainless steel hook-and-loop beige analog and digital round 2014 casio men's aw80v-5bv alarm-feature, water resistant, world time >>>> Watch features an analog/digital combination display, 30-record telememo, world time (30 cities), LED light, 1/100 second stopwatch, countdown timer, three daily alarms, hourly time signal, auto calendar, 12- and 24- hour formats, 10-year battery life and water resistance to 50 meters.


In [123]:
#Seems like its working. Now creating our dataframe for tfidf
X = dfcat[["Img_no","Final_String"]]

In [124]:
X.shape

(4957, 2)

In [125]:
# Lowercasing the st ring(aka normalizing)
X['Final_String'] = map(lambda x: x.lower(),X['Final_String'])

/Users/Iskandar/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [126]:
data = X.iloc[:,1]

### Tokenization and stop words removal


In [50]:
from nltk.corpus import stopwords
import nltk
nltk.download('all')

[nltk_data] Downloading collection u'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     /Users/Iskandar/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     /Users/Iskandar/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /Users/Iskandar/nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to
[nltk_data]    |     /Users/Iskandar/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to
[nltk_data]    |     /Users/Iskandar/nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to
[nltk_data]    |     /Users/Iskandar/nltk_data...
[nltk_data]    |   Package cess_cat is already up-to-da

False

In [127]:
import nltk.tokenize as nlt
#Toeknize
str_tokens = [nlt.word_tokenize(string) for string in data] 


In [128]:
str_tokens

[['black',
  'resin',
  'men',
  "'s",
  'standard',
  'resin',
  'casio',
  'resin',
  'buckle',
  'black',
  'analog',
  'round',
  '2014',
  'casio',
  'men',
  "'s",
  'mrw200h-1bct',
  'water',
  'resistant'],
 ['black',
  'calfskin',
  'mens',
  'standard',
  'brass',
  'timex',
  'brass',
  'buckle',
  'white',
  'analog',
  'round',
  '2009',
  'timex',
  'men',
  "'s",
  'easy',
  'reader',
  'watch',
  'measures-seconds',
  ',',
  'light'],
 ['black',
  'resin',
  'mens',
  'plastic',
  'casio',
  'plastic',
  'buckle',
  'white',
  'analog',
  'round',
  '2014',
  'casio',
  'men',
  "'s",
  'mq24-7b2',
  'analog',
  'watch',
  'with',
  'black',
  'resin',
  'band',
  'water',
  'resistant'],
 ['black',
  'resin',
  'mens',
  'resin',
  'casio',
  'resin',
  'buckle',
  'white',
  'analog',
  'round',
  'nan',
  'casio',
  'unisex',
  'mq24-7b',
  'analog',
  'black',
  'resin',
  'watch',
  'water',
  'resistant'],
 ['biege',
  'nylon',
  'mens-standard',
  'stainless',
  

In [129]:
#Remove Stop Words
new_tokens = []

cachedstopwords = set(stopwords.words('english'))#Cacheing makescode faster

for l in str_tokens:
    for word in cachedstopwords :
        #We keep updating l everytime we remove a list of words
        l =  filter(lambda x: x != word,l)
    new_tokens.append(l)

In [130]:
new_tokens[0]

['black',
 'resin',
 'men',
 "'s",
 'standard',
 'resin',
 'casio',
 'resin',
 'buckle',
 'black',
 'analog',
 'round',
 '2014',
 'casio',
 'men',
 "'s",
 'mrw200h-1bct',
 'water',
 'resistant']

### Stemming

In [131]:
from nltk.stem.snowball import SnowballStemmer
snowball = SnowballStemmer('english')


In [132]:
snow = []
#Token here represents a bag of words(list within a list)
for token in new_tokens:
    new_s  = map(lambda x:snowball.stem(x),token)
        #We keep updating l everytime we remove a list of words
    #print new_p   
    snow.append(new_s)
    

In [133]:
type(snow)

list

In [134]:
#reforming the corpus from this corrected words
word_data = []
for lis in snow:
    word_data.append(" ".join(lis))

In [135]:
type(word_data)

list

### TFIDF

In [136]:

vect = TfidfVectorizer(stop_words=stopwords.words('english'), norm='l2')
#Takes in a string
tfidf = vect.fit_transform(word_data).todense()



In [137]:
tfidf.shape, type(tfidf)

((4957, 7371), numpy.matrixlib.defmatrix.matrix)

### Merging the numerical and categorical parts of original dataframe again

In [138]:
#Adding columns 
X = np.concatenate((dfnum,tfidf),axis = 1)

In [139]:
X.shape, dfnum.columns

((4957, 7376), Index([           u'Band width', u'Water resistant depth',
                u'Case diameter',        u'Case Thickness',
                       u'Img_no'],
       dtype='object'))

In [140]:
#Making Image number the index beacuse its not a feature
dataframe = pd.DataFrame(X)

In [141]:
dataframe.head()

,0,1,2,3,4,5,6,7,8,9,...,7366,7367,7368,7369,7370,7371,7372,7373,7374,7375
0,22,330,43,12.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,18,99,35,8.0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,18,289,35,8.0,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,20,50,35,7.8,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,20,330,41,11.0,4,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [142]:
# Adding columsn names for our reference

colnames = list(dfnum.columns)
colnames.extend(vect.get_feature_names())

In [143]:
dataframe.columns = colnames

In [144]:
dataframe.head(13)

,Band width,Water resistant depth,Case diameter,Case Thickness,Img_no,00,000,001,001sl,002,...,zr15797,zr2001,zr2cs01,zr3005,zr66035,zr66076,zr77109,zr77242,zr77267,zulu
0,22,330,43.0,12.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,18,99,35.0,8.0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,18,289,35.0,8.0,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,20,50,35.0,7.8,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,20,330,41.0,11.0,4,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,20,330,37.0,13.4,5,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,24,165,39.5,12.0,6,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,20,289,30.0,8.0,7,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,16,165,38.0,10.0,8,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,20,99,40.0,9.0,9,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [145]:
# We can see above that image numbers(col 5) are not the same as index. 
#This is beacause dome images were not saved and hence those number are not here
#Creating a dictionary to remember this mapping
Index_dict = {}
for k,v in zip(dataframe.index,(dataframe.Img_no)):
    Index_dict.update({k : int(v)})
  

In [146]:
# Drop Image number
dataframe.drop(['Img_no'], axis =1 , inplace = True)

### Modeling

In [147]:
from sklearn.neighbors import NearestNeighbors

In [148]:
model_jac= NearestNeighbors(n_neighbors = 30,metric = 'jaccard') #Default is minkowlowski
model_cosine = NearestNeighbors(n_neighbors = 10,metric='cosine', algorithm = 'brute')
model_euc = NearestNeighbors(n_neighbors = 10,metric='euclidean')
model_man = NearestNeighbors(n_neighbors = 10,metric='chebyshev')


model_jac.fit(dataframe.values)
model_cosine.fit(dataframe.values)
model_euc.fit(dataframe.values) 
model_man.fit(dataframe.values) 


NearestNeighbors(algorithm='auto', leaf_size=30, metric='chebyshev',
         metric_params=None, n_jobs=1, n_neighbors=10, p=2, radius=1.0)

In [149]:
print dataframe.shape
#Take random image number 1546
test = dataframe.iloc[10,:].reshape(1,-1) #1 row 14000 columns
test.shape

(4957, 7375)


(1, 7375)

### Results

In [150]:
result_ind_jac = model_jac.kneighbors(test,return_distance=False)
result_ind_cosine = model_cosine.kneighbors(test,return_distance=False)
result_ind_euc = model_euc.kneighbors(test,return_distance=False)
result_ind_man = model_man.kneighbors(test,return_distance=False)

In [ ]:
print result_ind_jac
print result_ind_cosine
print result_ind_euc
print result_ind_man

In [ ]:
import matplotlib.pyplot as plt
import cv2


def get_results(result_indices, Index_dict, json_dump = None):
    '''
    Input:results from model, index dictionart
    Output:prints the recommended images for represented by the model 
    '''
    
    #Getting the image numbers
    recommendation = []
    img_numbers = []
    for result in result_indices[0]:
        img_numbers.append(Index_dict[result] )
    print img_numbers
    #Getting the data from Mongodb using connection. On the server we would probably use the json dump from mongodb
    #plt.subplot(5,2)
    i = 0    
    for img in img_numbers:
            #print img from disc
            image_path = "Data2/{0}/{0}.jpg".format(img)
            try:
                img=cv2.imread(image_path)
            except:
                print "Image read error"
                pass
            resized_image = cv2.resize(img, (500, 750))
            cv2.imshow('%d'%i,resized_image)
            cv2.waitKey(2500)
            i += 1
        #plt.show()
            
    
    

In [ ]:
#get_results(result_ind_jac,Index_dict)
get_results(result_ind_euc,Index_dict)
#get_results(result_ind_cosine,Index_dict)
#get_results(result_ind_man,Index_dict)

In [ ]:
cv2.destroyAllWindows()

Performing the test on multiple items has confirmed that Jaccardian distance gives the best results here
We will use only use that model 

In [ ]:
#Pickling the trained model for future use

import cPickle
# save the model
with open('Metadata_model.pkl', 'wb') as fid:
    cPickle.dump(model_jac, fid)    

      
# load it again
# with open('my_dumped_classifier.pkl', 'rb') as fid:
#     metadata_model = cPickle.load(fid)

In [ ]:
!pwd

In [ ]:
with open('TFIDF_vectorizer.pkl', 'wb') as fid:
    cPickle.dump(vect, fid)  

In [ ]:
with open('TFIDF_vectorizer_df.pkl', 'wb') as fid:
    cPickle.dump(dataframe, fid)  

In [151]:
with open('Metadat_Model_Index_Dictionary.pkl', 'wb') as fid:
    cPickle.dump(Index_dict, fid)  

NameError: name 'cPickle' is not defined